In [1]:
sealed trait Nat
final object Zero extends Nat
final case class Suc(n: Nat) extends Nat

sealed trait List[+A]
final object Nil extends List[Nothing]
final case class Cons[A](head: A, tail: List[A]) extends List[A]

defined trait Nat
defined object Zero
defined class Suc
defined trait List
defined object Nil
defined class Cons

In [2]:
def length[A]: List[A] => Int = {
    case Nil => 0
    case Cons(h, t) => 1 + length(t)
} 

defined function length

In [3]:
sealed trait Ring 
case object Zero extends Ring
case object One extends Ring
case class Elem(x: Int) extends Ring
case class Add(x: Ring, y: Ring) extends Ring
case class Mult(x: Ring, y: Ring) extends Ring


defined trait Ring
defined object Zero
defined object One
defined class Elem
defined class Add
defined class Mult

In [4]:
def toInt: Ring => Int = {
    case Zero => 0
    case One => 1
    case Elem(x) => x
    case Add(x, y) => toInt(x) + toInt(y)
    case Mult(x, y) => toInt(x) * toInt(y)
}

defined function toInt

In [5]:
sealed trait RingF[+A]
case object Zero extends RingF[Nothing]
case object One extends RingF[Nothing]
case class Elem(x: Int) extends RingF[Nothing]
case class Add[A](x: A, y: A) extends RingF[A]
case class Mult[A](x:A, y: A) extends RingF[A]



defined trait RingF
defined object Zero
defined object One
defined class Elem
defined class Add
defined class Mult

In [6]:
val expresion1 = Mult(Elem(4), Add(Elem(3), One))

expresion1: Mult[Product with Serializable with RingF[Product with Serializable with RingF[Nothing]]] = Mult(Elem(4), Add(Elem(3), One))

In [7]:
def evalToInt: RingF[Int] => Int = {
    x => x match {
        case Zero => 0
        case One => 1
        case Elem(x) => x
        case Add(x, y) => x + y
        case Mult(x, y) => x * y
    }
}



defined function evalToInt

In [8]:
trait Functor[F[_]] {
    def map[A,B](f: A => B): F[A] => F[B]
}

val ringFunctor = new Functor[RingF] {
    override def map[A, B](f: A => B): RingF[A] => RingF[B] = {
        case Zero => Zero
        case One => One
        case Elem(x) => Elem(x)
        case Add(x, y) => Add(f(x), f(y))
        case Mult(x, y) => Mult(f(x), f(y))
    }
}

defined trait Functor
ringFunctor: AnyRef with Functor[RingF] = ammonite.$sess.cmd7$Helper$$anon$1@61da3ddc

In [9]:
def liftInt: RingF[RingF[Int]] => RingF[Int] = {
    ringFunctor.map(evalToInt)
}

defined function liftInt

In [10]:
case class Fix[F[_]](value: F[Fix[F]] )
object Fix {
  def fix[F[_]](ff: F[Fix[F]]): Fix[F] = new Fix[F](ff)
  def unfix[F[_]]: Fix[F] => F[Fix[F]] = f => f.value
}

defined class Fix
defined object Fix

In [11]:
type Ring = Fix[RingF]
val zero = Fix[RingF](Zero)
val one =  Fix[RingF](One)
def elem: Int  => Ring = x => Fix[RingF](Elem(x))
def add: (Ring, Ring) => Ring = (x, y) =>  Fix[RingF](Add(x, y))
def mult: (Ring, Ring) => Ring = (x, y) =>  Fix[RingF](Mult(x, y))

defined type Ring
zero: Fix[RingF] = Fix(Zero)
one: Fix[RingF] = Fix(One)
defined function elem
defined function add
defined function mult

In [19]:
val s = add(elem(7), elem(7))

s: Ring = Fix(Add(Fix(Elem(7)), Fix(Elem(7))))

In [16]:
def cata: Fix[RingF] => Int = {
    x => evalToInt(ringFunctor.map(cata)(Fix.unfix(x)))
}


def f1[RingF[_], A]: Fix[RingF] => RingF[Fix[RingF]] = {
    x => Fix.unfix(x)
}



defined function cata
defined function f1

In [21]:
def cata[F[_], A](alg: F[A] => A)(implicit F: Functor[F]): Fix[F] => A = {
    x => alg((F.map(cata(alg))(Fix.unfix(x))))
}

defined function cata

In [20]:
cata(s)

res19: Int = 14